<a href="https://colab.research.google.com/github/HarolifeforScience/Green-House/blob/main/WordEmbeddingTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import io
import os 
import re 
import string
import shutil
import pathlib

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense as tfDense
from tensorflow.keras.layers import TextVectorization as tfVec
from tensorflow.keras.layers import GlobalAveragePooling1D,Embedding
import tensorflow_datasets
import matplotlib.pyplot  as plt 

#TextVectorization Ebene :

#1.standardisieren sie jede probe (normalerweise kleinschreibung + interpunktionsstreifen) 
#2. teilen sie jedes beispiel in teilzeichenfolgen (normalerweise wörter) auf. 
#3. kombinieren sie teilzeichenfolgen zu token (normalerweise ngrams) 
#4. indextoken (ordnen sie jedem token einen eindeutigen int-wert zu) 
#5. transformieren sie jede stichprobe mit diesem index entweder in einen vektor 
#von ints oder in einen dichten gleitkommavektor.

## import io
## import os
## .
## .
## .
## .
## import tensorflow
## import tensorflow_hub
## import tensorflow_datasets
## import matplotlib.pyplot

In [8]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset = tf.keras.utils.get_file("acLimdb_v1.tar.gz",url, 
                                  untar = True , cache_dir = '.' , 
                                  cache_subdir = '')
dataset_dir = os.path.join(os.path.dirname(dataset),'aclImdb')
os.listdir(dataset_dir)
# dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url, untar=True, cache_dir='.', cache_subdir='')
# save the data in folder called - "aclImdb"
# list directory
# os.listdir(dataset_dir)

['test', 'README', 'imdb.vocab', 'imdbEr.txt', 'train']

In [10]:
train_dir = os.path.join(dataset_dir,'train')
unsup_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(unsup_dir)
## shutil.rmtree(remove_dir)
## remove_dir = create path for unsup
## shutil.rmtree(remove_dir)

In [17]:
batch_size = 1024
seed = 123

train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size,
    validation_split = 0.2,
    subset = 'training',
    seed = seed
    )
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train',
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'validation',
    seed = seed
)
## batch_size = 1024
## seed = 123
## train_ds = tf.keras.utils.text_dataset_from_directory()
## val_ds = tf.keras.utils.text_dataset_from_directory()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [18]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print("text", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])
## for text_batch, label_batch in train_ds.take(1):
  ## for i in range(5):
    ## print(label_batch[i].numpy(), text_batch.numpy()[i])

text b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
Label 0
text b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "greas

In [19]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)
## AUTOTUNE = tf.data.AUTOTUNE

## train_ds = cache and prefetch train_ds
## val_ds = cache and prefetch val_ds

Hier brauche ich eine Erklärung (Embedding -->?)


In [20]:
embedding_layer = tf.keras.layers.Embedding(1000,5)
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

In [27]:
from numpy import string_
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  s_html = tf.strings.regex_replace(lowercase, '<br />' , ' ')
  return tf.strings.regex_replace(s_html, '[%s]' % re.escape(string.punctuation), '')

## def custom_standardization(input_data):
    ## lowercase = convert input_data to lowercase
    ## cleanData = use tf.string.regex_replace to replace <br /> tags from text
    ## returnData = clean punctuation using regex_replace and return it 

vocab_size = 10000
sequence_length = 100

vectorize_layer = tfVec(
        standardize = custom_standardization,
        max_tokens = vocab_size,
        output_mode = 'int',
        output_sequence_length = sequence_length 
        )

text_ds = train_ds.map(lambda x,y : x)
vectorize_layer.adapt(text_ds)


  

In [30]:
embedding_dim = 16
model = tf.keras.Sequential([
    vectorize_layer,
    Embedding(vocab_size,embedding_dim,name ='embedding'),
    GlobalAveragePooling1D(),
    tfDense(16,activation = 'relu'),
    tfDense(1)]
)


model.compile(optimizer='adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy']
              )
model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 15
)


Epoch 1/15
20/20 [==============================] - 5s 196ms/step - loss: 0.6921 - accuracy: 0.5028 - val_loss: 0.6902 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 81ms/step - loss: 0.6879 - accuracy: 0.5028 - val_loss: 0.6847 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 81ms/step - loss: 0.6800 - accuracy: 0.5028 - val_loss: 0.6746 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 80ms/step - loss: 0.6671 - accuracy: 0.5028 - val_loss: 0.6597 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 81ms/step - loss: 0.6485 - accuracy: 0.5028 - val_loss: 0.6395 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 2s 81ms/step - loss: 0.6240 - accuracy: 0.5214 - val_loss: 0.6144 - val_accuracy: 0.5390
Epoch 7/15
20/20 [==============================] - 2s 81ms/step - loss: 0.5943 - accuracy: 0.5948 - val_loss: 0.5859 - val_accuracy: 0.6054
Epoch 8/15
2

In [31]:
weigths = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [32]:
out_v = io.open('vectors.tsv','w',encoding='utf-8')
out_m = io.open('metadata.tsv','w',encoding='utf-8')

for index , word in enumerate(vocab):
  if index==0:
    continue #skip 0 ,it's  padding
  vec = weigths[index]
  out_v.write('\t'.join([str(x) for x in vec]) + '\n')
  out_m.write(word + "\n")
out_v.close()